# Sincpro Framework - Middleware System Examples

This notebook demonstrates the simple and agnostic middleware system in the Sincpro Framework.

## Philosophy

The middleware system follows the Sincpro Framework's core principles:
- **Simple**: Middleware is just a function `(dto: Any) -> Any`
- **Agnostic**: Framework doesn't dictate implementation details
- **Developer Control**: Complete freedom in middleware logic
- **No Over-Engineering**: Minimal abstraction, maximum flexibility

In [ ]:
# Setup: Import required modules
import time
from sincpro_framework import UseFramework, Feature, DataTransferObject

## Basic Middleware Example

Let's start with simple validation and enrichment middleware.

In [ ]:
# Define DTOs
class PaymentRequest(DataTransferObject):
    amount: float
    user_id: str
    description: str

class EnrichedPaymentRequest(DataTransferObject):
    amount: float
    user_id: str
    description: str
    timestamp: float
    validated: bool = True

class PaymentResponse(DataTransferObject):
    transaction_id: str
    status: str
    processed_amount: float

In [ ]:
# Simple validation middleware
def validate_payment(dto):
    """Validate payment and raise error if invalid"""
    if hasattr(dto, 'amount') and dto.amount <= 0:
        raise ValueError("Amount must be positive")
    if hasattr(dto, 'user_id') and not dto.user_id.strip():
        raise ValueError("User ID is required")
    return dto

# Enrichment middleware that transforms DTO
def add_timestamp_and_validation(dto):
    """Add timestamp and validation flag by creating new DTO"""
    if isinstance(dto, PaymentRequest):
        return EnrichedPaymentRequest(
            amount=dto.amount,
            user_id=dto.user_id,
            description=dto.description,
            timestamp=time.time(),
            validated=True
        )
    return dto

In [ ]:
# Setup framework with middleware
framework = UseFramework("payment_system", log_after_execution=False)
framework.add_middleware(validate_payment)
framework.add_middleware(add_timestamp_and_validation)

# Define feature that works with enriched DTO
@framework.feature(EnrichedPaymentRequest)
class ProcessPaymentFeature(Feature):
    def execute(self, dto: EnrichedPaymentRequest) -> PaymentResponse:
        # The feature receives the enriched DTO with timestamp and validation
        transaction_id = f"TXN_{dto.user_id}_{int(dto.amount * 100)}"
        
        return PaymentResponse(
            transaction_id=transaction_id,
            status="SUCCESS",
            processed_amount=dto.amount
        )

In [ ]:
# Test 1: Valid payment
print("=== Test 1: Valid Payment ===")
try:
    valid_request = PaymentRequest(
        amount=100.50,
        user_id="USER123",
        description="Coffee purchase"
    )
    
    result = framework(valid_request, PaymentResponse)
    print(f"✅ Success: {result.transaction_id}")
    print(f"   Status: {result.status}")
    print(f"   Amount: ${result.processed_amount}")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test 2: Invalid payment - negative amount
print("=== Test 2: Invalid Payment (Negative Amount) ===")
try:
    invalid_request = PaymentRequest(
        amount=-50.0,
        user_id="USER123",
        description="Invalid payment"
    )

    result = framework(invalid_request, PaymentResponse)
    print(f"✅ Unexpected success: {result.transaction_id}")
except Exception as e:
    print(f"❌ Validation Error (Expected): {e}")

In [ ]:
# Test 3: Invalid payment - empty user ID
print("=== Test 3: Invalid Payment (Empty User ID) ===")
try:
    invalid_request = PaymentRequest(
        amount=100.0,
        user_id="",
        description="No user payment"
    )
    
    result = framework(invalid_request, PaymentResponse)
    print(f"✅ Unexpected success: {result.transaction_id}")
except Exception as e:
    print(f"❌ Validation Error (Expected): {e}")

## Advanced Example: DTO Transformation Chain

This example shows how middleware can transform DTOs through multiple stages, demonstrating the key requirement from the feedback.

In [ ]:
# Define a chain of DTO transformations
class UserRequest(DataTransferObject):
    name: str
    age: int
    email: str

class ValidatedUserRequest(DataTransferObject):
    name: str
    age: int
    email: str
    email_validated: bool
    age_validated: bool

class EnrichedUserRequest(DataTransferObject):
    name: str
    age: int
    email: str
    email_validated: bool
    age_validated: bool
    user_type: str  # adult/minor
    processing_timestamp: float

class UserResponse(DataTransferObject):
    user_id: str
    welcome_message: str
    account_type: str

In [ ]:
# Middleware 1: Validation and DTO transformation
def validate_user_data(dto):
    """Validate user data and transform to ValidatedUserRequest"""
    if isinstance(dto, UserRequest):
        # Validate email
        email_valid = '@' in dto.email and '.' in dto.email
        if not email_valid:
            raise ValueError(f"Invalid email format: {dto.email}")
            
        # Validate age
        age_valid = 0 <= dto.age <= 120
        if not age_valid:
            raise ValueError(f"Invalid age: {dto.age}")
            
        # Transform to validated DTO
        return ValidatedUserRequest(
            name=dto.name,
            age=dto.age,
            email=dto.email,
            email_validated=email_valid,
            age_validated=age_valid
        )
    return dto

# Middleware 2: Enrichment and further DTO transformation
def enrich_user_data(dto):
    """Enrich validated data and transform to EnrichedUserRequest"""
    if isinstance(dto, ValidatedUserRequest):
        user_type = "adult" if dto.age >= 18 else "minor"
        
        return EnrichedUserRequest(
            name=dto.name,
            age=dto.age,
            email=dto.email,
            email_validated=dto.email_validated,
            age_validated=dto.age_validated,
            user_type=user_type,
            processing_timestamp=time.time()
        )
    return dto

In [ ]:
# Setup framework for user processing
user_framework = UseFramework("user_system", log_after_execution=False)
user_framework.add_middleware(validate_user_data)
user_framework.add_middleware(enrich_user_data)

# Feature that works with the final enriched DTO
@user_framework.feature(EnrichedUserRequest)
class CreateUserFeature(Feature):
    def execute(self, dto: EnrichedUserRequest) -> UserResponse:
        # Feature receives the fully enriched DTO
        user_id = f"USER_{dto.name.upper()}_{int(dto.processing_timestamp)}"
        
        welcome_msg = f"Welcome {dto.name}!"
        if dto.user_type == "minor":
            welcome_msg += " (Parental supervision required)"
            
        account_type = "premium" if dto.user_type == "adult" else "junior"
        
        return UserResponse(
            user_id=user_id,
            welcome_message=welcome_msg,
            account_type=account_type
        )

In [ ]:
# Test DTO transformation chain - Adult user
print("=== DTO Transformation Test: Adult User ===")
try:
    # Start with original UserRequest
    original_request = UserRequest(
        name="Alice",
        age=25,
        email="alice@example.com"
    )
    
    print(f"Original DTO type: {type(original_request).__name__}")
    print(f"Original data: {original_request}\n")
    
    # Framework processes through middleware chain and feature
    result = user_framework(original_request)
    
    print(f"✅ Processing successful!")
    print(f"User ID: {result.user_id}")
    print(f"Welcome: {result.welcome_message}")
    print(f"Account: {result.account_type}")
    
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test DTO transformation chain - Minor user
print("=== DTO Transformation Test: Minor User ===")
try:
    # Start with original UserRequest
    original_request = UserRequest(
        name="Bob",
        age=16,
        email="bob@example.com"
    )
    
    print(f"Original DTO type: {type(original_request).__name__}")
    print(f"Original data: {original_request}\n")
    
    # Framework processes through middleware chain and feature
    result = user_framework(original_request)
    
    print(f"✅ Processing successful!")
    print(f"User ID: {result.user_id}")
    print(f"Welcome: {result.welcome_message}")
    print(f"Account: {result.account_type}")
    
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test validation failure
print("=== DTO Transformation Test: Validation Failure ===")
try:
    # Start with invalid UserRequest
    invalid_request = UserRequest(
        name="Charlie",
        age=25,
        email="invalid-email"  # No @ or .
    )
    
    print(f"Original DTO type: {type(invalid_request).__name__}")
    print(f"Original data: {invalid_request}\n")
    
    # This should fail in validation middleware
    result = user_framework(invalid_request)
    print(f"✅ Unexpected success: {result}")
    
except Exception as e:
    print(f"❌ Validation Error (Expected): {e}")

## Custom Middleware Examples

Show different ways to implement middleware - functions, lambdas, and classes.

In [ ]:
# Example: Authentication middleware
def check_authentication(dto):
    """Simple authentication check"""
    if hasattr(dto, 'user_id'):
        # Simulate authentication check
        if dto.user_id.startswith('GUEST_'):
            raise PermissionError("Guest users not allowed for this operation")
    return dto

# Example: Logging middleware using lambda
log_requests = lambda dto: print(f"Processing DTO: {type(dto).__name__}") or dto

# Example: Rate limiting middleware using class
class RateLimitMiddleware:
    def __init__(self, max_requests=5):
        self.max_requests = max_requests
        self.request_count = 0
    
    def __call__(self, dto):
        self.request_count += 1
        if self.request_count > self.max_requests:
            raise Exception(f"Rate limit exceeded: {self.request_count}/{self.max_requests}")
        return dto

In [ ]:
# Demo framework with custom middleware
demo_framework = UseFramework("demo_system", log_after_execution=False)
demo_framework.add_middleware(log_requests)  # Lambda middleware
demo_framework.add_middleware(check_authentication)  # Function middleware
demo_framework.add_middleware(RateLimitMiddleware(max_requests=3))  # Class middleware

class DemoRequest(DataTransferObject):
    user_id: str
    action: str

@demo_framework.feature(DemoRequest)
class DemoFeature(Feature):
    def execute(self, dto: DemoRequest) -> str:
        return f"Executed {dto.action} for {dto.user_id}"

In [ ]:
# Test different middleware types
print("=== Custom Middleware Demo ===")

# Test 1: Valid user
for i in range(2):
    try:
        request = DemoRequest(user_id="USER123", action=f"action_{i+1}")
        result = demo_framework(request)
        print(f"✅ Request {i+1}: {result}")
    except Exception as e:
        print(f"❌ Request {i+1} failed: {e}")

print()

# Test 2: Guest user (should fail authentication)
try:
    guest_request = DemoRequest(user_id="GUEST_456", action="guest_action")
    result = demo_framework(guest_request)
    print(f"✅ Guest request: {result}")
except Exception as e:
    print(f"❌ Guest request failed (Expected): {e}")

print()

# Test 3: Rate limit (should fail after 3 requests total)
for i in range(3):
    try:
        request = DemoRequest(user_id="USER789", action=f"rate_test_{i+1}")
        result = demo_framework(request)
        print(f"✅ Rate test {i+1}: {result}")
    except Exception as e:
        print(f"❌ Rate test {i+1} failed: {e}")

## Key Takeaways

1. **Simple Function-Based**: Middleware is just a callable `(dto: Any) -> Any`
2. **DTO Transformation**: Middleware can transform between different Pydantic models
3. **Sequential Processing**: Middleware executes in the order they're added
4. **Error Handling**: Any middleware can halt processing by raising exceptions
5. **Developer Control**: Complete freedom in implementing middleware logic
6. **Zero Breaking Changes**: Framework remains backward compatible

The middleware system is designed to be agnostic and give developers complete control over their cross-cutting concerns while maintaining the framework's philosophy of simplicity.